In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pds
import os
import glob
from multiprocessing import dummy

pool = dummy.Pool(3)

files=glob.glob("/home/jeandet/Documents/DATA/BOOT_CUR_*C_*V.txt.csv.fixed.calibrated")

In [2]:
power_on_left_index=325000
power_on_right_index=350000

memory_wash_index={}
sw_start_index={}
for file in files:
    calibrated=pds.read_csv(file,sep="\t")
    
    avg=calibrated["CH2"].iloc[7000000:7000500].mean()
    memory_wash_index[file]=np.where(calibrated["CH2"].iloc[7000000:8000000]>=avg+30)[0][0]+7000000
    
    avg=calibrated["CH2"].iloc[10100000:10100500].mean()
    sw_start_index[file]=np.where(calibrated["CH2"].iloc[10300000:10800000]>=avg+30)[0][0]+10300000
    
    del calibrated

"""
18+1 pulses, each RMAP packet is limited to 4*4000 bytes
LFR FSW 3.0.0.22:
.text segment = 285136 bytes
.data segment = 4976 bytes

-> .text correspond to 17.821  packets or 17x16000 bytes packets plus 1x13136 bytes packet
-> .data correspond to 0.311 packet 

Auto detected before sw start
"""


SBM1_left_index=14100000
SBM1_right_index=14350000



In [3]:
def plot(df,left,right,sufix):
    if not os.path.isfile(file+sufix):
        ax=df.iloc[left:right].plot(figsize=(24,12))
        plt.ylabel('Current (mA)')
        plt.xlabel('Time (µs)')
        plt.savefig(file+sufix,dpi=1200,format='pdf')
        plt.close()

In [4]:
for file in files:
    if not os.path.isfile(file+".pdf"):
        calibrated=pds.read_csv(file,sep="\t")
        plot(calibrated,power_on_left_index,power_on_right_index,"_power_on.pdf")

        index=memory_wash_index[file]
        plot(calibrated,index-10000,index+150000,"_memory_wash.pdf")

        index=sw_start_index[file]
        plot(calibrated,index-320000,index-500,"_spw_upload.pdf")

        index=sw_start_index[file]
        plot(calibrated,index-2000,index+500000,"_sw_start.pdf")

        plot(calibrated,SBM1_left_index,SBM1_right_index,"_SBM1.pdf")

        plot(calibrated,0,-1,".pdf")
        del calibrated
